In [1]:
import torch
print(torch.__version__)

1.7.1


In [2]:
from __future__ import print_function

import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms
from torch.autograd import Variable
from torchvision import models
import numpy
import dataset
import random
from utils import *
from VGG_gru import FERANet
# from tensorboardX import SummaryWriter
import argparse
import subprocess
import math

In [3]:
# os.environ['CUDA_VISIBLE_DEVICES'] = "1"

description='PyTorch Facial Expression'
batch_size = 1
epochs = 20 #120
length = 32
preInitial = True
# dataset = 'Emotiw'

In [4]:
trn= "BDItrain1.txt"
vl= "BDIval1.txt"
tst= "BDItest1.txt"

In [5]:
trainlist = "/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/"+trn
vallist =  "/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/"+vl
testlist =  "/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/"+tst

In [6]:
print(trainlist)

/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/TrainTestlist/Emotiw/BDItrain1.txt


In [7]:
path_d= "/home/duaa/Desktop/GRU-test/emotion_classification-master/VGG_GRU/dataset.py"

In [8]:
if os.path.isfile != True:
	subprocess.call(["python", path_d])

In [9]:
# if os.path.isfile != True:
# 	subprocess.call(["python", "./TrainTestlist/"+dataset+"/getTraintest_"+dataset+".py"])

In [10]:
backupdir     = "/tmp/duaa/GRU"
batch_size    = 1
# learning_rate = 0.00001
learning_rate = 0.0001

# accuracy = 0.
# best_accuracy = 0.
metric = AccumulatedAccuracyMetric()

####here for same result#####
num_workers   = 0
# torch.backends.cudnn.enabled = False
torch.manual_seed(1)
torch.cuda.manual_seed(1)
random.seed(1)
torch.backends.cudnn.benchmark = True
# torch.backends.cudnn.deterministic = True

In [11]:
if preInitial == True:
	model = FERANet()
	model = Initial(model)
else:
	model = FERANet()

start loading vgg16 pre-fera2013 model...
finish loading vgg16 pre-fera2013 model!


In [12]:
processed_batches = 0
kwargs = {'num_workers': num_workers, 'pin_memory': True}
model = model.cuda()


optimizer = optim.SGD(model.parameters(), lr=learning_rate , momentum=0.9, weight_decay= 0.00005)

# loss_function = nn.CrossEntropyLoss()
loss_function = nn.L1Loss()
# loss_function = nn.MSELoss()

use_Tensorboard = False

In [13]:
def train(epoch,optimizer):


	train_loader = torch.utils.data.DataLoader(
		dataset.listDataset(trainlist,length = length,
					   shuffle=True,
					   train=True,
					   dataset = dataset),
		batch_size=batch_size, shuffle=False, **kwargs)


	for param_group in optimizer.param_groups:
		train_learning_rate = float(param_group['lr'])

	logging('epoch %d, processed %d samples, lr %f' % (epoch, epoch * len(train_loader.dataset), train_learning_rate))

	running_loss = 0.0

	model.train()

	for batch_idx, (data, label) in enumerate(train_loader):


		data = data.squeeze(0)
		data = Variable(data).cuda()

# 		label = Variable(label.long()).cuda()
		label = Variable(label.float()).cuda()
# 		label = label.squeeze(1)

		optimizer.zero_grad()

		output = model(data)

		loss = loss_function(output, label)

		running_loss += loss.data

		loss.backward()

		optimizer.step()

	if epoch %1 == 0:
		logging('Loss:{:.6f}'.format(running_loss))

In [14]:
def eval(epoch,metric):

	model.eval()

# 	global best_accuracy
# 	global accuracy

	metric.reset()
    
	accuracy = 0.
	best_accuracy = 0.
    
	test_loader = torch.utils.data.DataLoader(
		dataset.listDataset(vallist,length = length,
# 		dataset.listDataset(vallist,length = length,        
					shuffle=False,
					train=False,
					dataset = dataset),
					batch_size=1, shuffle=False, **kwargs)


	for batch_idx, (data, label) in enumerate(test_loader):

		data = data.squeeze(0)

		Batch,T,C,H,W = data.size()

		data = Variable(data,volatile=True).cuda()

		label = Variable(label.float(),volatile=True).cuda()
# 		label = Variable(label.long(),volatile=True).cuda()        
# 		label = label.squeeze(1)
# 		print('label')
		output = []
		for batch_index in range(Batch):
# 			print(batch_index)            
			output_feature = model(data[batch_index])

			output.append(output_feature)

		output = torch.mean(torch.cat(output), 0, keepdim=True)
        
# 		print("Dua's output is: ", output)
# 		metric(output, label) 
# # 		print(metric.value())

# 		accuracy, eval_loss = metric.value()
# # 		print("Dua's test is: ",accuracy)

        
# 	if accuracy >= best_accuracy:
# 		best_accuracy = accuracy
# 		print("saving accuracy is: ",accuracy)
# 		torch.save(model.state_dict(),'%s/model_%d.pkl' % (backupdir,epoch))

# 	logging("test accuracy: %f" % (accuracy))
# 	logging("best accuracy: %f" % (best_accuracy))

	return accuracy

In [15]:
def test(epoch,metric):

	model.eval()

# 	global best_accuracy
# 	global accuracy

	metric.reset()
    
	accuracy = 0.
	best_accuracy = 0.
    
	test_loader = torch.utils.data.DataLoader(
# 		dataset.listDataset(vallist,length = length,
		dataset.listDataset(testlist,length = length,        
					shuffle=False,
					train=False,
					dataset = dataset),
					batch_size=1, shuffle=False, **kwargs)


	for batch_idx, (data, label) in enumerate(test_loader):

		data = data.squeeze(0)

		Batch,T,C,H,W = data.size()

		data = Variable(data,volatile=True).cuda()

# 		label = Variable(label.long(),volatile=True).cuda()
		label = Variable(label.float(),volatile=True).cuda()
		label = label.squeeze(1)
# 		print('label')
		output = []
		for batch_index in range(Batch):
# 			print(batch_index)            
			output_feature = model(data[batch_index])

			output.append(output_feature)

		output = torch.mean(torch.cat(output), 0, keepdim=True)
        

# 		metric(output, label) 
# # 		print(metric.value())

# 		accuracy, eval_loss = metric.value()
# # 		print(accuracy)

        
# 	if accuracy >= best_accuracy:
# 		best_accuracy = accuracy
# # 		print("saving accuracy is: ",accuracy)
# # 		torch.save(model.state_dict(),'%s/model_%d.pkl' % (backupdir,epoch))

# 	logging("test accuracy: %f" % (accuracy))
# 	logging("best accuracy: %f" % (best_accuracy))

	return accuracy

In [16]:
for epoch in range(1, epochs+1): 

	train(epoch,optimizer)
	with torch.no_grad():
		eval_accuary = eval(epoch,metric)

2021-02-05 17:03:30 epoch 1, processed 36 samples, lr 0.000100
2021-02-05 17:03:42 Loss:338.842499


/home/duaa/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:28: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/duaa/anaconda3/envs/fastai/lib/python3.7/site-packages/ipykernel_launcher.py:30: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


2021-02-05 17:06:56 epoch 2, processed 72 samples, lr 0.000100
2021-02-05 17:07:09 Loss:335.152771
2021-02-05 17:10:26 epoch 3, processed 108 samples, lr 0.000100
2021-02-05 17:10:39 Loss:327.771423
2021-02-05 17:13:45 epoch 4, processed 144 samples, lr 0.000100
2021-02-05 17:13:57 Loss:319.840790
2021-02-05 17:16:53 epoch 5, processed 180 samples, lr 0.000100
2021-02-05 17:17:05 Loss:319.416931
2021-02-05 17:20:19 epoch 6, processed 216 samples, lr 0.000100
2021-02-05 17:20:31 Loss:312.565582
2021-02-05 17:23:44 epoch 7, processed 252 samples, lr 0.000100
2021-02-05 17:23:56 Loss:306.296875
2021-02-05 17:27:17 epoch 8, processed 288 samples, lr 0.000100
2021-02-05 17:27:30 Loss:299.341400
2021-02-05 17:30:32 epoch 9, processed 324 samples, lr 0.000100
2021-02-05 17:30:44 Loss:296.611664
2021-02-05 17:33:49 epoch 10, processed 360 samples, lr 0.000100
2021-02-05 17:34:03 Loss:288.498596
2021-02-05 17:37:08 epoch 11, processed 396 samples, lr 0.000100
2021-02-05 17:37:21 Loss:271.017273

KeyboardInterrupt: 

In [ ]:
for epoch in range(1, epochs+1): 
	with torch.no_grad():
		eval_accuary = test(epoch,metric)